# 3. Stepdetection
Lennard Rose
Start with Data_Exploration, contiue with Classify_Activities.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3')

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import utils
DEVICE = utils.get_device(cuda=True)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
print(os.listdir('./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/data'))

In [ ]:
BASE_PATH = "./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/"

In [ ]:
BASE_PATH = "./"

In [ ]:
OUT_PATH = BASE_PATH + "output/steps/"
DATA_PATH = BASE_PATH + "data/**/*.csv"
LABEL_PATH = BASE_PATH + "data/**/*.csv.stepMixed"
TEST_PATH = BASE_PATH + "data/testdata.csv"
RESNET_PATH = BASE_PATH + "models/rn_steps.pth"
CNN_PATH = BASE_PATH + "models/cnn_steps.pth"
LSTM_PATH = BASE_PATH + "models/lstm_steps.pth"

## ResNet
After the below networks did not yield any results I implemented this solution using a ResNet. I was curious if transforming the data to a frequency spectrum would do the trick. In my previous job in the navy, we always fourier transformed the acoustic signals we got from ships to classify them and to detect e.g. special engine sounds. Overall the idea worked out, because it was the first time I got an almost reasonable looking output and not just all ones or all zeros. The predictions are still mostly bad but it was a welcome surprise. In fact, I think before I can really utilize transformations like this, I will have to research them A LOT more. But it was a good first experience.
For this task I went with the classification of 0 and 1 for the steps and not for continous values.

In [ ]:
from sklearn.preprocessing import StandardScaler
from Data import get_SlidingWindow_Dataloader, get_balanced_dataset, load_dataframes
import utils

scaler = StandardScaler()
df = get_balanced_dataset(data_path=DATA_PATH, label_path= LABEL_PATH, method="expand", labels="start") # dont forget to set ordered_df
#df = load_dataframes(labels="start")

train_dl, val_dl, fitted_scaler = get_SlidingWindow_Dataloader(df=df,
                                                scaler=scaler,
                                                ordered_df=True,
                                                train_test_split=0.9,
                                                batch_size=128,
                                                shape_for="resnet",
                                                target_index="first")

Initialize Model

In [ ]:
from ResNet import ResNet

rn = ResNet(in_dim=600, out1_dim=256, out2_dim=128, final_out_dim=1, activity=False) # out = 1 because of binary

In [ ]:
import torch
rn.load_state_dict(torch.load(RESNET_PATH))

Train Model

For training the models I implemented an evaluation function that worked like the accuracy score but with a threshold of 10 around the steps. For dealing with the heavily inbalanced classes (~98% for 0 and ~2% for 1) I implemented an adjusted version of the mean squared error function that penalizes misclassifications of 1s.

In [ ]:
from training import train_model
from utils import hit_accuracy, save_model
from Custom_MSE import MSE_with_penalty
from torch import nn


criterion = nn.BCELoss()
# criterion = nn.MSELoss()
#criterion = MSE_with_penalty(penalty=50)

rn = rn.to(DEVICE)

trained_model, t_losses, t_accuracies, v_losses, v_accuracies = train_model(rn,
                                                                            learning_rate=0.01,
                                                                            criterion=criterion,
                                                                            epochs=50,
                                                                            training_dataloader=train_dl,
                                                                            test_dataloader=val_dl,
                                                                            device=DEVICE,
                                                                            path_to_save=RESNET_PATH,
                                                                            evaluationfunc=hit_accuracy,
                                                                            verbose=True)



The output copied from colab:
Note that the BCE loss usually yielded results like this. for the penalized MSE, the results started from around 98%, jumped down to 50% and eventually reaching around 70%

model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/rn_steps.pth
saved model
Epoch 1 / 50
Training Loss: 0.129 - Accuracy: 95.52%
Validation Loss: 0.104 - Accuracy: 97.86%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/rn_steps.pth
saved model
Epoch 2 / 50
Training Loss: 0.083 - Accuracy: 98.26%
Validation Loss: 0.103 - Accuracy: 97.86%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/rn_steps.pth
saved model
Epoch 3 / 50
Training Loss: 0.082 - Accuracy: 98.26%
Validation Loss: 0.102 - Accuracy: 97.86%
  ...
Epoch 49 / 50
Training Loss: 0.064 - Accuracy: 98.27%
Validation Loss: 0.124 - Accuracy: 97.86%
Epoch 50 / 50
Training Loss: 0.063 - Accuracy: 98.29%
Validation Loss: 0.131 - Accuracy: 97.86%

In [ ]:
from utils import save_model
save_model(trained_model, RESNET_PATH)

Predict Testset

In [ ]:
import pandas as pd
from Data import get_SlidingWindow_Dataloader
import numpy as np
import torch
from training import predict_steps

WINDOW_SIZE = 100
STEP_SIZE = 1

test_data = pd.read_csv(TEST_PATH)

test_data, _, _ = get_SlidingWindow_Dataloader(test_data,
                                               scaler=fitted_scaler,
                                                ordered_df=False,
                                                train=False,
                                                 batch_size=1,
                                                 window_size=WINDOW_SIZE,
                                                 step_size=STEP_SIZE,
                                                 train_test_split=None,
                                                  shape_for="resnet",
                                                target_index="first")

In [ ]:
result = predict_steps(trained_model, test_data, DEVICE)

In [ ]:
np.unique(result, return_counts=True)

As before, all cell outputs got lost when downloading this Notebook ( I tried to paste some interesting ones into the markdown cells here). Nevertheless, I have saved a few outputs to illustrate my general problem. The below outputs are after training this model for 500 epochs. I always ended up with one class being the mayority. I could overcome this to some extend with adjusting the penalty, but because the data and the models where so complex, I couldnt try every combination (even though I tried a lot).
Something I havent figured out is the amount of 0 and ones in those predictions. With the high accuracy you can see in the training above (~98%) I expected the model to ouput mostly zeros (because they are easy to hit) and only a couple of ones. I double checked the training and evaluation function to be sure they use the same method for rounding the values. But nevertheless I get mostly 1s when I predict. A possible answer to this may be that the outputs eventually converge pretty close to 0.5 and differ only by some hundreds. With the slight shift in data distribution in the test set, the outputs are shifted a bit "up" and get rounded to 1.

penalty 50 = (array([0, 1]), array([     1, 101990]))
BCE = (array([0, 1]), array([   391, 101600]))

In [ ]:
from utils import output_results

output_results(result, OUT_PATH + "rn_out.csv")

## CNN

Contrary to predicting the activities, the CNN was the hardest to train here. Every epoch took forever (4 minutes on GPU) and no real improvements could be seen. I tried some combinations but focussed more on the LSTM below.

In [ ]:
from Data import get_SlidingWindow_Dataloader, get_balanced_dataset, load_dataframes
import utils
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#df = get_balanced_dataset(data_path=DATA_PATH, label_path= LABEL_PATH, method="expand", labels="start") # cut expand SMOTE interpolate
df = load_dataframes(data_path=DATA_PATH, label_path= LABEL_PATH, labels="start")

train_dl, val_dl, fitted_scaler = get_SlidingWindow_Dataloader(df=df,
                                                                scaler = scaler,
                                                                ordered_df=True,
                                                                window_size=100, # default size is only thing implemented
                                                                step_size=1,
                                                                train_test_split=0.9,
                                                                batch_size=32,
                                                                shape_for="cnn",
                                                                target_index="first") # works only with first

Initialize Model

In [ ]:
from CNN import CNN

cnn = CNN(activity=False)

In [ ]:
import torch

cnn.load_state_dict(torch.load(CNN_PATH))

Train Model

In [ ]:
from training import train_model
from utils import hit_accuracy
from Custom_MSE import MSE_with_penalty

# criterion = nn.BCELoss()
# criterion = nn.MSELoss()
criterion = MSE_with_penalty(penalty=5)

cnn = cnn.to(DEVICE)
trained_model, t_losses, t_accuracies, v_losses, v_accuracies = train_model(cnn,
                                                                            learning_rate=0.01,
                                                                            criterion=criterion,
                                                                            epochs=20,
                                                                            training_dataloader=train_dl,
                                                                            test_dataloader=val_dl,
                                                                            device=DEVICE,
                                                                            patience_epochs=10,
                                                                            path_to_save=CNN_PATH,
                                                                            evaluationfunc=hit_accuracy,
                                                                            verbose=True)

In [ ]:
from utils import save_model

save_model(trained_model, CNN_PATH)


Predict Testset

In [ ]:
import pandas as pd
from Data import get_SlidingWindow_Dataloader
import numpy as np
import torch
from training import predict_steps

WINDOW_SIZE = 100
STEP_SIZE = 1

test_data = pd.read_csv(TEST_PATH)

test_data, _, _ = get_SlidingWindow_Dataloader(test_data,
                                               scaler=fitted_scaler,
                                                ordered_df=False,
                                                train=False,
                                                 batch_size=1,
                                                 window_size=WINDOW_SIZE,
                                                 step_size=STEP_SIZE,
                                                 train_test_split=None,
                                                  shape_for="cnn",
                                                target_index="first")
trained_model = cnn
result = predict_steps(trained_model, test_data, DEVICE)

np.unique(result, return_counts=True)

In [ ]:
from utils import output_results

output_results(result, OUT_PATH + "cnn_out.csv")

Again all 1s.
(array([1]), array([101991]))

## LSTM

In [ ]:
from Data import get_SlidingWindow_Dataloader, get_balanced_dataset, load_dataframes
import utils
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#df = get_balanced_dataset(method="cut")
df = load_dataframes(data_path=DATA_PATH, label_path= LABEL_PATH, labels="start")

train_dl, val_dl, fitted_scaler = get_SlidingWindow_Dataloader(df=df,
                                                scaler = scaler,
                                                ordered_df=False,
                                                train_test_split=0.9,
                                                batch_size=16,
                                                shape_for="lstm",
                                                target_index="n_to_n")

initialize Model

In [ ]:
from LSTM import LSTM

lstm = LSTM(input_size=6, hidden_size=128, num_layers=5, n_classes=1, activity=False) # n_classes = 1 because of binary

In [ ]:
import torch
lstm.load_state_dict(torch.load(LSTM_PATH))

Train Model

In [ ]:
# to continue training
trained_model = lstm

As you can see, the LSTM improved in terms of training loss over the time. The validation loss on the other hand did not decrease in the same amount. I simply had not enough time to train this so long to see real improvements or a "turningpoint".

In [ ]:
from training import train_model
from utils import hit_accuracy
from Custom_MSE import MSE_with_penalty


# criterion = nn.BCELoss()
# criterion = nn.MSELoss()
criterion = MSE_with_penalty(penalty=10)


lstm = lstm.to(DEVICE)
trained_model, t_losses, t_accuracies, v_losses, v_accuracies = train_model(lstm,
                                                                            criterion=criterion,
                                                                            learning_rate=0.001,
                                                                            epochs=10,
                                                                            patience_epochs=10,
                                                                            training_dataloader=train_dl,
                                                                            test_dataloader=val_dl,
                                                                            device=DEVICE,
                                                                            evaluationfunc=hit_accuracy,
                                                                            path_to_save=LSTM_PATH,
                                                                            verbose=True)

model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 1 / 10
Training Loss: 0.073 - Accuracy: 95.51%
Validation Loss: 0.123 - Accuracy: 97.12%
Epoch 2 / 10
Training Loss: 0.07 - Accuracy: 95.66%
Validation Loss: 0.126 - Accuracy: 96.44%
Epoch 3 / 10
Training Loss: 0.067 - Accuracy: 95.61%
Validation Loss: 0.125 - Accuracy: 96.7%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 4 / 10
Training Loss: 0.064 - Accuracy: 95.75%
Validation Loss: 0.121 - Accuracy: 96.75%
Epoch 5 / 10
Training Loss: 0.06 - Accuracy: 95.85%
Validation Loss: 0.121 - Accuracy: 96.91%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 6 / 10
Training Loss: 0.057 - Accuracy: 96.01%
Validation Loss: 0.117 - Accuracy: 96.94%
Epoch 7 / 10
Training Loss: 0.054 - Accuracy: 96.16%
Validation Loss: 0.118 - Accuracy: 97.08%
Epoch 8 / 10
Training Loss: 0.051 - Accuracy: 96.35%
Validation Loss: 0.119 - Accuracy: 96.84%
Epoch 9 / 10
Training Loss: 0.048 - Accuracy: 96.51%
Validation Loss: 0.121 - Accuracy: 96.95%
Epoch 10 / 10
Training Loss: 0.046 - Accuracy: 96.71%
Validation Loss: 0.12 - Accuracy: 97.02%

In [ ]:
from utils import save_model
save_model(trained_model, LSTM_PATH)

Predict Testset

In [ ]:
import pandas as pd
from Data import get_SlidingWindow_Dataloader
import numpy as np

WINDOW_SIZE = 100
STEP_SIZE = 50

test_data = pd.read_csv(TEST_PATH)

test_data, _, _ = get_SlidingWindow_Dataloader(test_data,
                                            scaler=fitted_scaler,
                                            ordered_df=False,
                                            train=False,
                                            batch_size=1,
                                            window_size=WINDOW_SIZE,
                                            step_size=50,
                                            train_test_split=None,
                                            shape_for="lstm",
                                            target_index="n_to_n")


In [ ]:
len(test_data.dataset)

In [ ]:
from training import predict_steps_average

result = predict_steps_average(trained_model, test_data, DEVICE)
np.unique(result, return_counts=True)

In [85]:
import torch
lstm.load_state_dict(torch.load(LSTM_PATH))

<All keys matched successfully>

## Conclusion
Overall, I learned a lot working on this assignment. I was forced to look far beyond the standard repertoire for possible solutions. In the process, I found some new methods that I didn't know yet. Even though it may not seem like much, it was beneficial to see how easy it is to implement an custom loss function to manipulate your results. I also learned and relearned a lot of theory to really make sure that the insufficient output was not due to a misimplementation on my part. The classification of activities, although not explicitly part of the task, was a rewarding experience, because I struggled at first but then it worked out very well. Nevertheless, I have mixed feelings, not only because the task was very (very) frustrating at times. But because I think that by making it difficult to achieve any results, the chance to find new ways to improve one's results was lost. I would have liked to try out my original plan with the activities as well as some ensemble methods. But in the end there was no time because I was too busy changing parameters, pressing a button and waiting 30 minutes to see if it would produce any useful results. Nevertheless, I am very satisfied overall with my performance and the portfolio. In contrast to my portfolio from the bachelor's, there was less (or no) cross-validation of hyperparameters this time. But instead, there are many different approaches that I would not have tried otherwise. In retrospect, I probably would have been better off joining one of the other teams. The overhead that came with working on the tasks alone was sometimes higher than expected. If I had had more time, I would have tried to replace the individual steps with blocks (20) to put the unbalanced class ratio into perspective. I would also have tried the continuous steps approach.

Train Model

In [82]:
# to continue training
trained_model = lstm

In [86]:
from training import train_model
from utils import hit_accuracy
from Custom_MSE import MSE_with_penalty


# criterion = nn.BCELoss()
# criterion = nn.MSELoss()
criterion = MSE_with_penalty(penalty=10)


lstm = lstm.to(DEVICE)
trained_model, t_losses, t_accuracies, v_losses, v_accuracies = train_model(lstm,
                                                                            criterion=criterion,
                                                                            learning_rate=0.001,
                                                                            epochs=10,
                                                                            patience_epochs=10,
                                                                            training_dataloader=train_dl,
                                                                            test_dataloader=val_dl,
                                                                            device=DEVICE,
                                                                            evaluationfunc=hit_accuracy,
                                                                            path_to_save=LSTM_PATH,
                                                                            verbose=True)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 1 / 10
Training Loss: 0.073 - Accuracy: 95.51%
Validation Loss: 0.123 - Accuracy: 97.12%
Epoch 2 / 10
Training Loss: 0.07 - Accuracy: 95.66%
Validation Loss: 0.126 - Accuracy: 96.44%
Epoch 3 / 10
Training Loss: 0.067 - Accuracy: 95.61%
Validation Loss: 0.125 - Accuracy: 96.7%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 4 / 10
Training Loss: 0.064 - Accuracy: 95.75%
Validation Loss: 0.121 - Accuracy: 96.75%
Epoch 5 / 10
Training Loss: 0.06 - Accuracy: 95.85%
Validation Loss: 0.121 - Accuracy: 96.91%
model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth
saved model
Epoch 6 / 10
Training Loss: 0.057 - Accuracy: 96.01%
Validation Loss: 0.117 - Accuracy: 96.94%
Epoch 7 / 10
Training Loss: 0.054 - Accuracy: 96.16%
Validation Loss: 0.118 - Accuracy: 97.08%
Epoch 8 / 10
Trainin

In [87]:
from utils import save_model
save_model(trained_model, LSTM_PATH)

model saved to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/models/lstm_steps.pth


Predict Testset

In [116]:
import pandas as pd
from Data import get_SlidingWindow_Dataloader
import numpy as np

WINDOW_SIZE = 100
STEP_SIZE = 50

test_data = pd.read_csv(TEST_PATH)

test_data, _, _ = get_SlidingWindow_Dataloader(test_data,
                                            scaler=fitted_scaler,
                                            ordered_df=False,
                                            train=False,
                                            batch_size=1,
                                            window_size=WINDOW_SIZE,
                                            step_size=50,
                                            train_test_split=None,
                                            shape_for="lstm",
                                            target_index="n_to_n")


In [117]:
len(test_data.dataset)

2040

In [118]:
from training import predict_steps_average

result = predict_steps_average(trained_model, test_data, DEVICE)
np.unique(result, return_counts=True)

/content/drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/training.py:239: RuntimeWarning: Mean of empty slice
  print(i)


2039


(array([0., 1.]), array([    41, 102050]))

In [121]:
from utils import output_results

output_results(result, OUT_PATH + "lstm_out.csv")

successfully saved results to ./drive/MyDrive/Colab_Notebooks/Master/SS23_Assignment_3/output/steps/lstm_out.csv
